In [2]:
import ipywidgets as ipyw
import json
import patatmo
import folium

In [3]:
# your patatmo connect developer credentials
credentials = json.load(open("C:\\users\\annefou\\Documents\\.netatmo.credentials"))

# create an api client
client = patatmo.api.client.NetatmoClient()
# tell the client's authentication your credentials
client.authentication.credentials = credentials
# optionally give the authentication a temporary file.
# The tokens are then stored there for later reuse,
# e.g. next time you invoke this script.
# This saves time because no new tokens have to be requested.
# New tokens are then only requested if the old ones expire.
client.authentication.tmpfile = "temp_auth.json"


In [4]:

# lat/lon outline of Hamburg/Germany
oslo_region = {
    "lat_ne" : 59.9999,
    "lat_sw" : 59.7809,
    "lon_ne" : 10.76,
    "lon_sw" : 10.70,
}

# issue the API request
oslo = client.Getpublicdata(region = oslo_region)
# convert the response to a pandas.DataFrame
print(oslo.dataframe())

                     altitude  humidity                 id   latitude  \
id                                                                      
70:ee:50:12:c3:38  209.000000      82.0  70:ee:50:12:c3:38  59.962199   
70:ee:50:1b:0c:22  252.000000      83.0  70:ee:50:1b:0c:22  59.962722   
70:ee:50:28:ef:12  174.000000      87.0  70:ee:50:28:ef:12  59.959788   
70:ee:50:02:39:be  183.105591       NaN  70:ee:50:02:39:be  59.961331   
70:ee:50:05:65:38  208.000000     100.0  70:ee:50:05:65:38  59.964518   
70:ee:50:1b:06:76         NaN       NaN  70:ee:50:1b:06:76  59.961346   
70:ee:50:27:2b:d2         NaN     100.0  70:ee:50:27:2b:d2  59.962381   
70:ee:50:00:3a:6a  189.000000      90.0  70:ee:50:00:3a:6a  59.960436   
70:ee:50:28:ad:7e  171.000000      88.0  70:ee:50:28:ad:7e  59.967889   
70:ee:50:2a:df:90         NaN      90.0  70:ee:50:2a:df:90  59.963683   
70:ee:50:15:ec:58  242.000000      92.0  70:ee:50:15:ec:58  59.968448   
70:ee:50:19:0c:de  191.000000      78.0  70:ee:50:1

In [6]:
latitudes=oslo.dataframe()['latitude']
longitudes=oslo.dataframe()['longitude']
temperatures=oslo.dataframe()['temperature']
for lat,lon,t in zip(latitudes, longitudes, temperatures):
    print(lat,lon,t)

59.9621995 10.6894028 -1.1
59.9627216 10.7059413 0.2
59.959787860335 10.729260685145 -0.8
59.961331 10.726771 nan
59.9645181 10.7363399 -0.2
59.961346 10.751003 nan
59.962381 10.736948 -1.4
59.9604358 10.7622564 -0.9
59.967889 10.770193 -1.2
59.963683 10.760551 -1.5
59.968448 10.796146 -2.2
59.9605696 10.7939425 0.1
59.9615836 10.781494 nan
60.034953 10.80468 -1.5
60.038232 10.803251 -1.8
59.970197895381 10.813395565082 -2.6
59.979139 10.817572 -2.8
60.039672 10.805705 -2.6
60.067594 10.827687 -3.1
60.0592673 10.8510214 -1.5
60.093228020301 10.836133686475 -4.5
59.9644078 10.8633102 -2.3
59.96063841001 10.863991635582 -1.5
59.9654245 10.8695242 -1.3
60.049405846454 10.860368860852 nan
60.0502854 10.8593161 -1.6
60.049421440321 10.860154327509 -1.0
60.0560695 10.8634356 -1.6
60.0599942 10.8564152 nan
60.058950677382 10.858346415456 -1.9
60.084776467674 10.864900528033 -1.5
60.08754 10.873741 -0.9
60.089301 10.869684 -0.9
60.089932 10.865328 0.0
59.964201656947 10.883369938402 -1.3
59.96

In [7]:
def df_to_geojson(df, properties, lat='latitude', lon='longitude'):
    geojson = {'type':'FeatureCollection', 'features':[]}
    for _, row in df.iterrows():
        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'Point',
                               'coordinates':[]}}
        feature['geometry']['coordinates'] = [row[lon],row[lat]]
        for prop in properties:
            feature['properties'][prop] = row[prop]
        geojson['features'].append(feature)
    return geojson

In [8]:
cols = ['temperature', 'altitude', 'humidity']
geojson = df_to_geojson(oslo.dataframe(), cols)
geojson

{'features': [{'geometry': {'coordinates': [10.6894028, 59.9621995],
    'type': 'Point'},
   'properties': {'altitude': 209.0, 'humidity': 82.0, 'temperature': -1.1},
   'type': 'Feature'},
  {'geometry': {'coordinates': [10.7059413, 59.9627216], 'type': 'Point'},
   'properties': {'altitude': 252.0, 'humidity': 83.0, 'temperature': 0.2},
   'type': 'Feature'},
  {'geometry': {'coordinates': [10.729260685145, 59.959787860335],
    'type': 'Point'},
   'properties': {'altitude': 174.0, 'humidity': 87.0, 'temperature': -0.8},
   'type': 'Feature'},
  {'geometry': {'coordinates': [10.726771, 59.961331], 'type': 'Point'},
   'properties': {'altitude': 183.105591, 'humidity': nan, 'temperature': nan},
   'type': 'Feature'},
  {'geometry': {'coordinates': [10.7363399, 59.9645181], 'type': 'Point'},
   'properties': {'altitude': 208.0, 'humidity': 100.0, 'temperature': -0.2},
   'type': 'Feature'},
  {'geometry': {'coordinates': [10.751003, 59.961346], 'type': 'Point'},
   'properties': {'al

In [13]:
lon, lat = 10.7522, 59.9139

mapa = folium.Map(location=[lat, lon], zoom_start=10, tiles='Stamen Terrain')

dy = 0.01

for lat,lon,t in zip(latitudes, longitudes, temperatures):
    info_to_plot = str(t) + " Celcius"
    folium.Marker([lat, lon-2*dy], popup=folium.Popup(info_to_plot)).add_to(mapa)
mapa

In [14]:
print(type(oslo.dataframe()))

<class 'pandas.core.frame.DataFrame'>


In [19]:
dfs = oslo.dataframe().sort_values(by=['time_temperature'])

In [20]:
dfs

,altitude,humidity,id,latitude,longitude,pressure,temperature,time_humidity,time_pressure,time_temperature,timezone
id,,,,,,,,,,,
70:ee:50:19:db:1a,-25.000000,88.0,70:ee:50:19:db:1a,59.722912,10.732377,986.1,-0.3,2018-01-05 08:14:26+00:00,2018-01-05 08:15:06+00:00,2018-01-05 08:14:26+00:00,Europe/Oslo
70:ee:50:29:14:d4,NaN,98.0,70:ee:50:29:14:d4,60.039672,10.805705,958.0,-2.6,2018-01-05 08:35:09+00:00,2018-01-05 08:35:36+00:00,2018-01-05 08:35:09+00:00,Europe/Oslo
70:ee:50:20:9b:c0,101.000000,83.0,70:ee:50:20:9b:c0,59.943650,10.727249,990.1,-0.2,2018-01-05 08:47:42+00:00,2018-01-05 10:59:34+00:00,2018-01-05 08:47:42+00:00,Europe/Oslo
70:ee:50:04:80:ac,220.000000,93.0,70:ee:50:04:80:ac,59.956848,10.938169,986.9,-2.0,2018-01-05 10:40:01+00:00,2018-01-05 10:56:39+00:00,2018-01-05 10:40:01+00:00,Europe/Oslo
70:ee:50:27:9b:06,149.000000,87.0,70:ee:50:27:9b:06,60.089301,10.869684,994.6,-0.9,2018-01-05 10:43:23+00:00,2018-01-05 10:44:05+00:00,2018-01-05 10:43:23+00:00,Europe/Oslo
70:ee:50:19:2c:fc,39.000000,70.0,70:ee:50:19:2c:fc,59.905149,10.793615,992.5,0.4,2018-01-05 10:45:26+00:00,2018-01-05 11:04:38+00:00,2018-01-05 10:45:26+00:00,Europe/Oslo
70:ee:50:29:0f:a6,0.000000,89.0,70:ee:50:29:0f:a6,59.828372,10.760409,1000.9,0.6,2018-01-05 10:54:20+00:00,2018-01-05 10:54:22+00:00,2018-01-05 10:54:20+00:00,Europe/Oslo
70:ee:50:04:70:b6,147.000000,88.0,70:ee:50:04:70:b6,59.789499,10.812890,987.7,-1.1,2018-01-05 10:54:27+00:00,2018-01-05 10:55:05+00:00,2018-01-05 10:54:27+00:00,Europe/Oslo
70:ee:50:29:0d:94,NaN,92.0,70:ee:50:29:0d:94,59.732534,10.937917,973.8,-1.9,2018-01-05 10:54:32+00:00,2018-01-05 10:55:15+00:00,2018-01-05 10:54:32+00:00,Europe/Oslo
